Reference
- hangman I got pygame from https://www.pygame.org/project/4116
- hand tracking I got library from https://github.com/google/mediapipe/blob/master/docs/solutions/hands.md#min_detection_confidence
- figer tracking code guideline, My friend got from https://www.section.io/engineering-education/creating-a-finger-counter/

In [1]:
import pygame, sys, random, cv2

from time import sleep
from timeit import default_timer as timer
import time

import numpy as np
import mediapipe as mp
import os
from pygame.locals import *

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as tf_img
import matplotlib.pyplot as plt

pygame 2.0.1 (SDL 2.0.14, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Users/Kirl2oyale/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
## add more function 

def box_dim(text1, x=30, y=30, fontScale= 2, thickness=2): #check text dimension 
    (x_t1, y_t1), _ = cv2.getTextSize(text1, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    dummy = (x_t1, y_t1)
    box_w, box_h = dummy[0] + fontScale, dummy[1] + fontScale
    box_x, box_y = x - fontScale, y - dummy[1] - fontScale
    return (x, y),(box_x, box_y), (box_w, box_h)
    
    
def draw_box(image, text='Submit', x = 180 , y=30, color_text=(0,0,0), color_rect=(9, 121, 105)):
    (x, y),(box_x,box_y), (box_w, box_h) = box_dim(text, x, y)
    image = cv2.rectangle(image, (box_x,box_y), (x+box_w, y+box_h), color_rect, cv2.FILLED)
    image = cv2.putText(image, text, (x, y+ int(box_h/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text, 1) #(x,y) of cv2.putText method is bottomleft recgtangular
    return image

def fingerUp(landmark_list):  #function check which fingertip is up
    f_up = []
    fingerTip = [4, 8, 12, 16, 20]
    for i in range(5):
        if i == 0:
            # thump
            if landmark_list[fingerTip[i]][1] > landmark_list[fingerTip[i] - 1][1]:
                f_up.append(1)
            else:
                f_up.append(0)
        else:
            # not thump
            if landmark_list[fingerTip[i]][2] < landmark_list[fingerTip[i] - 2][2]:
                f_up.append(1)
            else:
                f_up.append(0)
    return f_up 


def draw(letter_canvas, x0, y0, x1, y1, color=(227, 11, 92),thickness = 5):
    if (x0, y0) == (0, 0):
        (x1, y1) = (x0, y0)
    else:
        cv2.line(letter_canvas, (x0, y0), (x1, y1), color, thickness)
            
            
def mode(landmark_list, fingertip_up, img,x1, y1,x0,y0,letter_canvas): 
    fingerTip = [4, 8, 12, 16, 20]
    x_finger = list(map(lambda x: x[1], landmark_list))
    y_finger = list(map(lambda x: x[2], landmark_list))
    name = ''
    if fingertip_up[1] & fingertip_up[2] & fingertip_up[3] and sum(fingertip_up) == 3:
        name = 'Erase'
        color = (0, 0, 0)
        cv2.circle(img,(x_finger[fingerTip[1]], y_finger[fingerTip[1]]), 10, (255, 255, 255), cv2.FILLED)
        draw(letter_canvas, x0, y0, x1, y1, color,thickness = 10)
    elif fingertip_up[1] & fingertip_up[2] and sum(fingertip_up) == 2:
        name = 'Select'
        pass
    elif fingertip_up[0] & fingertip_up[1] & fingertip_up[4] & (1-fingertip_up[2]) & (1-fingertip_up[3]):
        name = 'Clear'
        letter_canvas[:,:,:] = 0
    elif fingertip_up[1] and sum(fingertip_up) == 1:
        name = 'Draw'
        color =(227, 11, 92)
        cv2.circle(img, (x_finger[fingerTip[1]], y_finger[fingerTip[1]]), 5, (227, 115, 131) , cv2.FILLED)
        draw(letter_canvas, x0, y0, x1, y1, color)
    return name

def clear_text(canvas):
    canvas[:,:,:] = 0
    return canvas


def guess(letter_canvas, predict_canvas, cond, pred):
    word_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',
                 14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}
    if cond:
        canvas_box = cv2.flip(letter_canvas, 1)[10:100, 30:120]
        canvas_box[np.any(canvas_box!=[0,0,0], axis=-1)] = [255,255,255]
            ############ save image ############## 
#        if not os.path.exists('./result'):
#            os.mkdir(f'./result')
#        pic_name = len(os.listdir('./result')) + 1
#        cv2.imwrite(f'./result/{pic_name}_box.png', canvas_box)
            ###################################
        canvas_box = cv2.resize(canvas_box, (64, 64))
        canvas_box = np.expand_dims(canvas_box, axis = 0)
        pred = word_dict[model.predict(canvas_box).argmax()]
        predict_canvas = clear_text(predict_canvas)
        predict_canvas = cv2.putText(predict_canvas, f'{pred}?', (50,150), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.7, (250, 218, 94), 2) 
    return predict_canvas, pred



def submit(predict_canvas,letter_canvas,cond, predicted_letter):
    if cond:
        predict_canvas = clear_text(predict_canvas)
        predict_canvas = cv2.putText(predict_canvas, f'{predicted_letter} sent.', (35,150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (9, 121, 105), 2) 
        letter_canvas = clear_text(letter_canvas)
        guess_letter = predicted_letter.lower()
    return predict_canvas, letter_canvas, guess_letter

In [3]:
fps = 30
pygame.init()
width = 800
height = 600

black = (0,0,0)
white = (255,255,255)
lightred = (255, 165, 145)
darklightred = (255, 97, 81)
lightblue = (126,178,255)
darklightblue = (42, 129, 255)
lightgrey = (192, 192, 192)

textBoxSpace = 5
textBoxNumber = 0

def button(word,x,y,w,h,ic,ac,action=None):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()

    if x+w > mouse[0] > x and y+h > mouse[1] > y:
        pygame.draw.rect(screen,ac,(x,y,w,h))
        if click[0] == 1 and action != None:
            action()
    else:
        pygame.draw.rect(screen,ic,(x,y,w,h))

    buttonText = pygame.font.Font("freesansbold.ttf",20)
    buttonTextSurf = buttonText.render(word, True, white)
    buttonTextRect = buttonTextSurf.get_rect()
    buttonTextRect.center = ((x+(w/2)), (y+(h/2)))
    screen.blit(buttonTextSurf, buttonTextRect)

def endGame():
    global textBoxSpace, textBoxNumber, end, start
    end = timer()
    print("Time it took: ",end - start)
    timeTaken = (end - start)
    textBoxSpace = 5
    textBoxNumber = 0
    message = "Time taken: " + str(round(timeTaken)) + "s"
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
        button("Yes",(width/2)-50,420,100,50,darklightred,lightred,quitGame)
        button("No",(width/2)-50,500,100,50,darklightred,lightred,hangman)
        
        largeText = pygame.font.SysFont("comicsansms",115)
        TextSurf = largeText.render("End Game?",True,darklightred)
        TextRect = TextSurf.get_rect()
        TextRect.center = (width / 2, height / 2)
        screen.blit(TextSurf, TextRect)

        textSurf = largeText.render(message,True,darklightred)
        textRect = textSurf.get_rect()
        textRect.center = (width/2,200)
        screen.blit(textSurf, textRect)
        
        pygame.display.update()
        clock.tick(fps)
        
def quitGame():
    pygame.quit()
    sys.exit()

def unpause():
    global pause
    pause = False

def pause():
    largeText = pygame.font.SysFont("comicsansms",115)
    TextSurf = largeText.render("Paused",True,black)
    TextRect = TextSurf.get_rect()
    TextRect.center = (width / 2, height / 2)
    screen.blit(TextSurf, TextRect)

    while pause:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
        screen.fill(white)
        

        button("Continue",150,450,100,50,darklightred,lightred,unpause)
        button("Quit",550,450,100,50,darklightblue,lightblue,quitgame)

        pygame.display.update()
        clock.tick(fps)

def textObjects(text, font):
    textSurface = font.render(text, True, black)
    return textSurface, textSurface.get_rect()
    
def main():
    global clock, screen, play
    play = True
    clock = pygame.time.Clock()
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Hangman!")

    while True:
        hangman()

def placeLetter(letter):
    global pick, pickSplit
    space = 10
    wordSpace = 0
    while wordSpace < len(pick):
        text = pygame.font.Font('freesansbold.ttf',40)
        if letter in pickSplit[wordSpace]:
            textSurf = text.render(letter,True,black)
            textRect = textSurf.get_rect()
            textRect.center = (((150)+space),(200))
            screen.blit(textSurf, textRect)
        wordSpace += 1
        space += 60

    pygame.display.update()
    clock.tick(fps)
        
def textBoxLetter(letter):
    global textBoxSpace, textBoxNumber
    if textBoxNumber <= 5:
        text = pygame.font.Font("freesansbold.ttf",40)
        textSurf = text.render(letter,True,black)
        textRect = textSurf.get_rect()
        textRect.center = (((105)+textBoxSpace),(350))
        screen.blit(textSurf, textRect)

    elif textBoxNumber <= 10:
        text = pygame.font.Font("freesansbold.ttf",40)
        textSurf = text.render(letter,True,black)
        textRect = textSurf.get_rect()
        textRect.center = (((105)+textBoxSpace),(400))
        screen.blit(textSurf, textRect)

    elif textBoxNumber <= 15:
        text = pygame.font.Font("freesansbold.ttf",40)
        textSurf = text.render(letter,True,black)
        textRect = textSurf.get_rect()
        textRect.center = (((105)+textBoxSpace),(450))
        screen.blit(textSurf, textRect)

    elif textBoxNumber <= 20:
        text = pygame.font.Font("freesansbold.ttf",40)
        textSurf = text.render(letter,True,black)
        textRect = textSurf.get_rect()
        textRect.center = (((105)+textBoxSpace),(500))
        screen.blit(textSurf, textRect)  
        
    pygame.display.update()
    clock.tick(fps)

def hangman():
    global textBoxSpace, textBoxNumber
    textBoxSpace = 5
    textBoxNumber = 0
    while play == True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
        screen.fill(white)
        space = 10
        textBoxSpace = 5
        
        text = pygame.font.Font("freesansbold.ttf",20)
        textSurf = text.render("Choose a catagory",True,black)
        textRect = textSurf.get_rect()
        textRect.center = ((width/2),(height/2))
        screen.blit(textSurf, textRect)

        button("Animals",150,450,150,100,black,lightgrey,Animals)
        button("Vehicles",550,450,150,100,black,lightgrey,Vehicles)
        button("Food",150,50,150,100,black,lightgrey,Foods)
        button("Sports",550,50,150,100,black,lightgrey,Sports)

        pygame.display.update()
        clock.tick(fps)

def hangmanGame(catagory,title):
    global pause, pick, pickSplit, textBoxSpace, textBoxNumber, start, fingerTip, model
    start = timer()
    chances = 20
    pick = random.choice(catagory)
    pickSplit = [pick[i:i+1] for i in range(0, len(pick), 1)]
    
    #### add some parameter for hand tracking ####
    x0 = y0 = 0
    counter = 0
    guess_lst = []
    fingerTip = [4, 8, 12, 16, 20]
    letter_canvas = np.zeros((170, 270, 3), np.uint8)
    predict_canvas = np.zeros((170, 270, 3), np.uint8)

    _,(box_x,box_y), (box_w, box_h) = box_dim('Guess!', x = 180, y =30)
    box_guess = np.array([[box_x-4, box_x+box_w+4], [box_y-4, box_y+box_h+4]])
    _,(box_x,box_y), (box_w, box_h) = box_dim('Submit', x = 180, y =80)
    box_submit = np.array([[box_x-4, box_x+box_w+4], [box_y-4, box_y+box_h+4]])
    
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)
    mpDraw = mp.solutions.drawing_utils
    
    model = load_model('./model_vgg16_text.h5',compile=False)

    ##############################
    
    screen.fill(white)
    
    wordSpace = 0
    space = 10
    while wordSpace < len(pick):
        text = pygame.font.Font("freesansbold.ttf",40)
        textSurf1 = text.render("_",True,black)
        textRect1 = textSurf1.get_rect()
        textRect1.center = (((150)+space),(200))
        screen.blit(textSurf1, textRect1)
        space = space + 60
        wordSpace += 1
            
    guesses = ''
    gamePlay = True
    while gamePlay == True:
        #guessLett = ''

        if textBoxNumber == 5:
            textBoxSpace = 5
        if textBoxNumber == 10:
            textBoxSpace = 5
        if textBoxNumber == 15:
            textBoxSpace = 5

        
        #pygame.draw.rect(screen, white, [550,20,200,20]) # move white rect (using for revising change 20 >> change 19) to middle
        pygame.draw.rect(screen, white, [250,80,320,40])
        text = pygame.font.Font("freesansbold.ttf",20)
        textSurf = text.render(("Chances: %s" % chances),False,black)
        textRect = textSurf.get_rect()
        #textRect.topright = (700,20) # move "change xx" from right to middle (want some space for webcam on right conner)#
        textRect.topright = (415,100)
        screen.blit(textSurf, textRect)

        textTitle = pygame.font.Font("freesansbold.ttf",40)
        textTitleSurf = textTitle.render(title,True,black)
        textTitleRect = textTitleSurf.get_rect()
        textTitleRect.center = (350,50)  #title name 
        screen.blit(textTitleSurf, textTitleRect)

        pygame.draw.rect(screen, black, [100,300,250,250],2)

        if chances == 19:
            pygame.draw.rect(screen,black,[450,550,100,10])
        elif chances == 18:
            pygame.draw.rect(screen,black,[550,550,100,10])
        elif chances == 17:
            pygame.draw.rect(screen,black,[650,550,100,10])
        elif chances == 16:
            pygame.draw.rect(screen,black,[500,450,10,100])
        elif chances == 15:
            pygame.draw.rect(screen,black,[500,350,10,100])
        elif chances == 14:
            pygame.draw.rect(screen,black,[500,250,10,100])
        elif chances == 13:
            pygame.draw.rect(screen,black,[500,250,150,10])
        elif chances == 12:
            pygame.draw.rect(screen,black,[600,250,100,10])
        elif chances == 11:
            pygame.draw.rect(screen,black,[600,250,10,50])
        elif chances == 10:
            pygame.draw.line(screen,black,[505,505],[550,550],10)
        elif chances == 9:
            pygame.draw.line(screen,black,[550,250],[505,295],10)
        elif chances == 8:
            pygame.draw.line(screen,black,[505,505],[460,550],10)
        elif chances == 7:
            pygame.draw.circle(screen,black,[605,325],30)
        elif chances == 6:
            pygame.draw.rect(screen,black,[600,350,10,60])
        elif chances == 5:
            pygame.draw.rect(screen,black,[600,410,10,60])
        elif chances == 4:
            pygame.draw.line(screen,black,[605,375],[550,395],10)
        elif chances == 3:
            pygame.draw.line(screen,black,[605,375],[650,395],10)
        elif chances == 2:
            pygame.draw.line(screen,black,[605,465],[550,485],10)
        elif chances == 1:
            pygame.draw.line(screen,black,[605,465],[650,485],10)

        button("Back",50,50,100,50,black,lightgrey,hangman)
        
        #################### connected webcam ####################
        bg = pygame.Surface((270,170))
        cap = cv2.VideoCapture(0)
        _, img = cap.read()
        img = cv2.resize(img,(270,170)) # webcam size
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        predict_letter = ""
        guess_letter = ""
        
        results = hands.process(img)
        if results.multi_hand_landmarks:
            landmark_list = [] 
            for handLms in results.multi_hand_landmarks:
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS) # add 20 landmarks to img
                for idx, lm in enumerate(handLms.landmark): # return landmarks to landmark_list
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    landmark_list.append([idx] + [cx, cy]) #return 20 indexs(idx) and locations(cx,cy) of hand lanmarks ex. [[[0, 84, 143], [1, 107, 147]]
            fingertip_up = fingerUp(landmark_list) # return what fingertip is up
            x1, y1 = landmark_list[fingerTip[1]][1:]
            mode_name = mode(landmark_list, fingertip_up, img,x1, y1,x0,y0,letter_canvas)
            cond_guess = (box_guess[0][0] <= (270-x1) <= box_guess[0][1]) and (box_guess[1][0] <= y1 <= box_guess[1][1]) and (mode_name == 'Select')
            if cond_guess:
                predict_canvas, predicted_letter = guess(letter_canvas, predict_canvas,cond_guess,predict_letter)
            cond_submit = (box_submit[0][0] <= (270-x1) <= box_submit[0][1]) and (box_submit[1][0] <= y1 <= box_submit[1][1]) and (mode_name == 'Select')
            if cond_submit:
                 predict_canvas, letter_canvas, guess_letter = submit(predict_canvas,letter_canvas,cond_submit, predicted_letter)
            x0, y0 = x1, y1
            
        img = cv2.flip(img, 1)
        img = cv2.rectangle(img, (30,10), (30+90, 10+90), (52, 52, 52), 3) #adding drawing box
        img = draw_box(img, text='Guess!', color_rect = (250, 218, 94)) #adding guess box
        img = draw_box(img, text='Submit', x=180, y=80) #adding select box
        canvas = cv2.flip(letter_canvas, 1) + predict_canvas
        img[np.any(canvas != [0, 0, 0], axis=-1)] = canvas[np.any(canvas != [0, 0, 0], axis=-1)]
        frame = img.swapaxes(0,1)
        pygame.surfarray.blit_array(bg,frame)
        screen.blit(bg,(530,0)) # webcam location
        pygame.display.flip()      
        pygame.display.update()
        
        ################ display guess letter #################        
        if guess_letter not in guesses:
            guesses += guess_letter
            if guess_letter in pick:
                placeLetter(f'{guess_letter}')
                counter += 1
                if counter == len(set(pick)):
                    endGame()
            else:
                textBoxSpace += 40
                textBoxNumber += 1
                chances = chances - 1
                textBoxLetter(f'{guess_letter}')
            if chances == 0:
                endGame()                    
        #######################################################
        
        pygame.display.update()
        clock.tick(fps)

    pygame.display.update()
    clock.tick(fps)

def Animals():
    animal = ['cow','dog','cat','pig','zebra','bird','giraffe','lion','tiger','penguin','hamster','fox','panda','bear','cheetah','ostrich','meerkat','whale','shark','horse','monkey','octopus','kitten','kangaroo','chicken','fish','rabbit','sheep']
    print("animal")
    title = "Animals"
    hangmanGame(animal,title)

def Vehicles():
    vehicle = ['car','bus','train','airplane','plane','ship','jet','boat','lorry','tractor','bike','motorbike','tram','van','ambulance','rocket','taxi','caravan','coach','lorry','scooter','sleigh','tank','wagon','spaceship']
    print("vehicle")
    title = "Vehicles"
    hangmanGame(vehicle,title)
    
def Foods():
    food = ['apple','banana','orange','peach','pizza','donut','chips','sandwich','cookie','cucumber','carrot','sweetcorn','pancake','bread','potato','tomato','nuts','yogurt','pasta','rice','cheese','soup','fish','egg','meat','ham','sausage','tomyam','padtai']
    print("food")
    title = "Foods"
    hangmanGame(food,title)
    
def Sports():
    sport = ['rugby','football','netball','basketball','swimming','hockey','curling','running','golf','tennis','badmington','archery','volleyball','bowling','dancing','gym','skating','baseball','rounders','boxing','climbing','canoe','cycling','fencing','karate','shooting','cricket']
    print("sport")
    title = "Sports"
    hangmanGame(sport,title)

    
if __name__ == '__main__':
    main()
            

food
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
gue

guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: s
guess_letter1: s
guess_letter1: s
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess

guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_letter1: 
guess_le

SystemExit: 

/Users/Kirl2oyale/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
